In [1]:
from grand import topography
from grand.tools.topography import Topography, Reference

from grand.tools import ECEF, Geodetic, LTP, GRAND, CartesianRepresentation
import numpy as np

In [2]:
# update data
loc = Geodetic(latitude=45, longitude=90, height=1000)
topography.update_data(coordinates=loc)
#topography.update_data(coordinates=loc, radius=1e3) # radius in meter

In [19]:
# elevation and undulation with respect to ellipsoid model and mean sea level (geoid).
# elevation : distance of topography from ellisoid (default)
# undulation: distance of geoid from ellipsoid
# elevation wrt geoid = elevation wrt ellipsoid - undulation

elev_1= topography.elevation(loc, Reference.ELLIPSOID) # elevation wrt ellipsoid
elev_2= topography.elevation(loc, Reference.GEOID)     # elevation wrt geoid
print('Elevation w.r.t ellipsoid and geoid at a given location.')
print('    elevation wrt ellip :', elev_1) 
print('    elevation wrt geoid :', elev_2, '\n') 

# Change datadir based on your grand location.
datadir = '/Users/rameshkoirala/Documents/GRAND/grand/grand/tools/data/topography'
topo = Topography(datadir)
elev1 = topo.elevation(loc) # method 1
elev2= topography.elevation(loc) # method 2
print('Elevation w.r.t ellipsoid (default) at a given location.')
print('    elevation method1  :', elev1)
print('    elevation method2  :', elev2) 

undu = topography.geoid_undulation(loc)        # distance of geoid from ellipsoid.
print('    undulation         :', undu) 

# Elevation wrt ellipsoid.
elev_wrt_ellip = topography.elevation(loc) - topography.geoid_undulation(loc) # =elev1+undu. being explicit here.
print('    elevation wrt geoid:', elev_wrt_ellip, '(elevation wrt ellipsoid - undulation)\n')

# elevation based on LTP and GRAND coordinate system.
ltp  = LTP(x=0, y=0, z=0, location=loc, magnetic=True, orientation='NWU') 
gcs1 = GRAND(x=0, y=0, z=0, location=loc)     # z-coordinate does not matter. elevation wrt to XY-plane at (x,y).
gcs2 = GRAND(x=100, y=100, z=0, location=loc) # z-coordinate does not matter. elevation wrt to XY-plane at (x,y).
print('Elevation based on GRAND coordinate system.')
print('    elevation LTP    :', topography.elevation(ltp))
print('    elevation GRAND1 :', topography.elevation(gcs1))
print('    elevation GRAND2 :', topography.elevation(gcs2), '\n')




Elevation w.r.t ellipsoid and geoid at a given location.
    elevation wrt ellip : 1015.3753332956809
    elevation wrt geoid : 1008.0 

Elevation w.r.t ellipsoid (default) at a given location.
    elevation method1  : 1015.3753332956809
    elevation method2  : 1015.3753332956809
    undulation         : 7.375333295680863
    elevation wrt geoid: 1008.0 (elevation wrt ellipsoid - undulation)

Elevation based on GRAND coordinate system.
    elevation LTP    : 15.375333295588277
    elevation GRAND1 : 15.375333295588277
    elevation GRAND2 : 20.091694868170485 



In [24]:
# topography distance. distance(position, direction, maximum_distance).
# Position should be in known coordinate system, i.e. Geodetic, ECEF, GRAND, or LTP. Internally it will
#    be transformed to ECEF.
# direction is a vector pointing towards the particle trajectory. Internally unit vector is calculated and used.
#    Error occurs if the direction is not converted to an unit vector.
# masimum_distance uses 0 or large number. Large number has to be larger than the distance being calculated. 
# Example: maximum_distance = np.array([1e6]). default=0
direction = CartesianRepresentation(x=1, y=-2, z=3) # Random vector
position  = loc
distance1 = topography.distance(position, direction) # method 1
distance2 = topo.distance(position, direction)       # method 2
print('Distance using random vector.')
print('    distance method1:', distance1)
print('    distance method2:', distance2, '\n')

# Distance in this case should be equal to height-elevation. Height of a position and elevation at that lat, lon.
direction = ECEF(position)
distance1 = topography.distance(position, direction) # method 1
distance2 = topo.distance(position, direction)       # method 2
print("Direction vector pointing towards the position from Earth's center.")
print("By construction (to test), distance = height-elevation = %.2f - %.2f"%(loc.height,topo.elevation(loc)))
print('    distance method1:', distance1)
print('    distance method2:', distance2, '\n')

Distance using random vector.
    distance method1: -104.79553357465221
    distance method2: -104.79553357465221 

Direction vector pointing towards the position from Earth's center.
By construction (to test), distance = height-elevation = 1000.00 - 1015.38
    distance method1: -15.373746979222032
    distance method2: -15.373746979222032 

